adapted from :https://medium.com/@chand.shelvin/pybullet-getting-started-a068a0e3d492

http://www.mymodelrobot.appspot.com/5674976526991360

https://wiki.ros.org/urdf/XML/link

https://www.programcreek.com/python/example/122099/pybullet.setJointMotorControl2

In [99]:
import pybullet as p
import time
import pybullet_data
import math
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [93]:
import random as r
from urdfpy import URDF
import urdfpy
import numpy as np

In [52]:
for i in range(0,8,2):
   physicsClient = p.connect(p.GUI)# p.DIRECT for non-graphical version
   p.setAdditionalSearchPath(pybullet_data.getDataPath()) # optionally
   p.setGravity(0,0,-9.8)   
   planeId = p.loadURDF('plane.urdf')
   cubeStartPos = [0,0,1]
   cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
   # boxId = p.loadURDF('Models/Model_24.urdf',cubeStartPos, cubeStartOrientation)

   boxId = p.loadURDF('STARFISH2.0.urdf',cubeStartPos, cubeStartOrientation)
   # p.setJointMotorControl2(boxId,0,p.POSITION_CONTROL,-1)
   # p.setJointMotorControl2(boxId,1,p.POSITION_CONTROL,-1)
   
   p.setJointMotorControl2(boxId,(i+1),p.POSITION_CONTROL,-1.57)
   
   
   cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
   # print(cubePos,cubeOrn)
   p.resetDebugVisualizerCamera( cameraDistance=3.5, cameraYaw=30, cameraPitch=52, cameraTargetPosition=cubePos)
   for i in range (300):
      p.stepSimulation()
      time.sleep(1./200.)
   cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
   print(cubePos,cubeOrn)
   p.disconnect()

(0.00031439727782053633, -0.020131218691226187, 0.46698282496745397) (0.11071366557071198, 1.4306856382682968e-05, 0.00012585081957326537, 0.9938523372276092)
(-0.0002833563121864383, 0.020099923695879058, 0.46697881508630684) (-0.1107156715211334, 5.1359029875874616e-05, 0.00012322408656520903, 0.9938521128707727)
(-0.020099633433193962, -0.00038686368601430974, 0.4669986483811819) (0.00010303414843149941, -0.1107057358071606, 0.00013083062332979652, 0.9938532146784591)
(0.020107879904122654, 0.0002880717024984953, 0.4670239428236519) (7.442984885374358e-05, 0.11069317401764413, 0.00011865797329432217, 0.9938546179428772)


In [37]:
# 4 legged starfish.[[0.10038424283224191, -2.5319705873951107e-05, -0.0004969729635007379, 0.9949486198632488],[-0.10038085877465497, -2.5816714159629625e-05, 0.0005083451955946118, 0.9949489555300424],[1.9557973627556364e-05, -0.10061859289800221, 0.0004570184810641759, 0.994924966776299],[-2.9342634783195095e-05, 0.1007576192581765, 0.0002754417050166459, 0.9949109635702591]]
[list(0.11071366557071198, 1.4306856382682968e-05, 0.00012585081957326537, 0.9938523372276092),list(-0.1107156715211334, 5.1359029875874616e-05, 0.00012322408656520903, 0.9938521128707727),list(0.00010303414843149941, -0.1107057358071606, 0.00013083062332979652, 0.9938532146784591),list(7.442984885374358e-05, 0.11069317401764413, 0.00011865797329432217, 0.9938546179428772)]

p.disconnect()

In [89]:
def create_base_link(xyz_rpy):
    col = urdfpy.Collision(name=None,origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy),geometry=urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None))
    vis = urdfpy.Visual(urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None),material=urdfpy.Material("red",[0.969, 0.129, 0.122,1]))    
    return urdfpy.Link("base_link",collisions=[col],inertial=None,visuals=[vis])

def create_arm_link(name,xyz_rpy):
    col = urdfpy.Collision(name=None,origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy),geometry=urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None))
    vis = urdfpy.Visual(urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None),origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy))    
    return urdfpy.Link(name,collisions=[col],inertial=urdfpy.Inertial(1,[[0, 0, 0],[0, 0, 0],[0, 0, 0]],urdfpy.xyz_rpy_to_matrix(xyz_rpy)),visuals=[vis])

def create_joint(parent,child,xyz_rpy,name,joint_type):
    child.origin = xyz_rpy
    return urdfpy.Joint(name,joint_type=joint_type,parent=parent.name,child=child.name,limit=urdfpy.JointLimit(1.57,1.57,-1.57,1.57),origin=xyz_rpy)

def create_appendige(base,xyz_rpy,name):
    arm1 = create_arm_link(name+"_arm1",[0, .34, 0,0,0,0])
    joint_base = create_joint(base,arm1,xyz_rpy,name+"_jointBase","fixed")
    arm2 = create_arm_link(name+"_arm2",[0, .34, 0,0,0,0])
    joint1 = create_joint(arm1,arm2,[0, .68, 0,0,0,0],name+"_joint1","revolute")
    return [arm1,arm2],[joint_base,joint1]

base = create_base_link([0,0,0,0,0,0])

print()

links,joints = create_appendige(base,[0,.34,0,0,0,0],"set1")
links2,joints2 = create_appendige(base,[0,-.34,0,0,0,3.141],"set2")
links3,joints3 = create_appendige(base,[.34,0,0,0,0,-1.57],"set3")
links4,joints4 = create_appendige(base,[-.34,0,0,0,0,1.57],"set4")

robot = URDF("STARFISH2.0",links=links+links2+links3+links4+[base], joints=joints+joints2+joints3+joints4)
robot.save("Models/STARFISH2.0.urdf")


In [17]:
arm1 = create_arm_link("test",[0,.34,0,0,0,0])
base = create_base_link([0,0,0,0,0,0])

# robot = URDF("test",links=[base,arm1], joints=[joint1])
# robot.save("robotTest.urdf")

In [64]:
def generateModels(amountOfModelsToBeGenerated, amountOfMotorOnEachModel):
    for i in range(0,amountOfModelsToBeGenerated):
        jointList = []
        linksList = []
        base = create_base_link([0,0,0,0,0,0])
        linksList.append(base)
        for j in range(amountOfMotorOnEachModel):
            d = 0
            x = 0
            y = 0
            z = 0
            while(d<=.5):
                x = r.randrange(-50,50,1)/100
                y = r.randrange(-50,50,1)/100
                z = r.randrange(-10,10,1)/100
                # print(x," ",y," ",z)
                d = math.sqrt((x**2)+(y**2)+(z**2))

            yaw = 0
            roll = 0
            pitch = 0

            yaw = (r.randrange(100,200,1)/100)
            roll = (r.randrange(-100,100,1)/100)
            pitch = 0

            if(x>0):
                yaw = -yaw 
                roll = -roll
            
            # orientationVector = np.asarray([0,0,0])
            # while (np.dot(orientationVector, [0,0,0])>0): # not sure if this works
            #     pitch = r.uniform(-3.14159,3.14159) 
            #     yaw = r.uniform(-3.14159,3.14159) 
            #     roll = r.uniform(-3.14159,3.14159)
            #     orientationVector = np.asarray([pitch,yaw,roll])

            links,joints = create_appendige(base,[x,y,z,roll,pitch,yaw],"arm_"+str(j))
            jointList.extend(joints)
            linksList.extend(links)
        robot = URDF("Model_"+str(i),links=linksList, joints=jointList)
        robot.save("Models/Model_"+str(i)+".urdf")
# generateModels(100,2)

In [106]:
def accuracyOfModel(amountOfMotors, modelNameTrue, modelNamePredicted):
    model = URDF.load(modelNameTrue)
    listTrue = []
    for j in range(0, amountOfMotors*2,2):
        # print(list(urdfpy.matrix_to_xyz_rpy(model.joints[j].origin))) # get these values then add or subtract random values 100 times and see results
        listTrue.append(list(urdfpy.matrix_to_xyz_rpy(model.joints[j].origin)))

    model = URDF.load(modelNamePredicted)
    PredictedJoints = []
    for j in range(0, amountOfMotors*2,2):
        # print(list(urdfpy.matrix_to_xyz_rpy(model.joints[j].origin))) # get these values then add or subtract random values 100 times and see results
        PredictedJoints.append(list(urdfpy.matrix_to_xyz_rpy(model.joints[j].origin)))

    mse = []
    for k in range(0,len(listTrue)):
        # for l in range(0, len(listTrue[k])):
        mse.append(cosine_similarity(np.array(listTrue[k]).reshape(1,-1),np.array(PredictedJoints[k]).reshape(1,-1))[0,0])
    # mse = sum(mse)/2.0

    return sum(mse)/float(amountOfMotors)

accuracyOfModel(2, "STARFISH2.0.urdf","Models/Model_76.urdf")


0.6875925415201581

In [115]:
def simulateModels(amountToSimulate, compareOrientations, motoramount):
    bestList = []
    for i in range(0,amountToSimulate):
        bestList.append(["Model_"+str(i)])
    # print(bestList)
    
    currentBest = ["na",100]
    for model in bestList:
        best2 = [[],[],[],[]] # this needs to be dynamic not good rn
        auxCounter = 0
        for i in range(0,motoramount*2,2):
            physicsClient = p.connect(p.DIRECT)# p.DIRECT for non-graphical version
            p.setAdditionalSearchPath(pybullet_data.getDataPath()) # optionally
            p.setGravity(0,0,-9.8)   
            planeId = p.loadURDF('plane.urdf')
            cubeStartPos = [0,0,1]
            cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
            boxId = p.loadURDF('Models/'+str(model[0])+'.urdf',cubeStartPos, cubeStartOrientation)
            # boxId = p.loadURDF('starfish.urdf',cubeStartPos, cubeStartOrientation)
            # p.setJointMotorControl2(boxId,0,p.POSITION_CONTROL,-1)
            # p.setJointMotorControl2(boxId,1,p.POSITION_CONTROL,-1
            p.setJointMotorControl2(boxId,(i+1),p.POSITION_CONTROL,-1)
            cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
            # print("start")
            # print(cubePos,cubeOrn)
            # p.resetDebugVisualizerCamera( cameraDistance=3.5, cameraYaw=30, cameraPitch=52, cameraTargetPosition=cubePos)
            for i in range (300):
                p.stepSimulation()
                # time.sleep(1./200.)
            cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)

            mean = mean_squared_error(compareOrientations[auxCounter],list(cubeOrn))
    
            best2[auxCounter] = mean
            # print(cubePos,cubeOrn
            # print(mean, model[0])
            
            p.disconnect()  
            auxCounter+=1
        # print(best2)
        if(sum(best2)<currentBest[1]):
            currentBest[1] = sum(best2)
            currentBest[0] = model[0]
            # currentBest[2] = accuracyOfModel(2, "STARFISH2.0.urdf","Models/"+model[0]+".urdf")

    return currentBest

simulateModels(500, [[0.11071366557071198, 1.4306856382682968e-05, 0.00012585081957326537, 0.9938523372276092],[-0.1107156715211334, 5.1359029875874616e-05, 0.00012322408656520903, 0.9938521128707727],[0.00010303414843149941, -0.1107057358071606, 0.00013083062332979652, 0.9938532146784591],[7.442984885374358e-05, 0.11069317401764413, 0.00011865797329432217, 0.9938546179428772]],4)

['Model_14', 0.0009757795869712971]

In [114]:
def generateModelsStep(amountOfModelsToBeGenerated, amountOfMotorOnEachModel, randomConst, randomConst2, randomConst3):
    pool = [[0,.34,0],[.34,.34,0],[-.34,-.34,0],[-.34,0,0],[.34,0,0],[0,-.34,0],[.34,-.34,0],[-.34,.34,0]]
    
    for i in range(0,amountOfModelsToBeGenerated):  
        jointList = []
        linksList = []
        base = create_base_link([0,0,0,0,0,0])
        linksList.append(base)
        mountTaken = [] 
        for j in range(amountOfMotorOnEachModel):
            x = 0
            y = 0
            z = 0
            currentMount = pool[r.randrange(0,len(pool))]
            while((currentMount in mountTaken)):
                currentMount = pool[r.randrange(0,len(pool))]

            mountTaken.append(currentMount)

            x= currentMount[0]
            y= currentMount[1] #apply noise
            z = 0 # r.randrange(-10,10,1)/100

            yaw = 0
            pitch = 0
            roll = 0
            
            if(y>0 and x>0):
                yaw = (r.randrange(-157,314,1)/100) 

            elif(y<0 and x<0):
                yaw = (r.randrange(0,157,1)/100)

            elif(y<0 and x>0):
                yaw = (r.randrange(0,157,1)/100)

            elif(y>0 and x<0):
                yaw = (r.randrange(157,314,1)/100)

            elif(y>0 and x==0):
                yaw = (r.randrange(-100,100,1)/100)

            elif(y==0 and x>0):
                yaw = (r.randrange(-100,-200,-1)/100)

            elif(y<0 and x==0):
                yaw = (r.randrange(214,414,1)/100)

            elif(y==0 and x<0):
                yaw = (r.randrange(100,200,1)/100)
            

            if (randomConst!=0 and randomConst2!=0 and randomConst3!=0 ):
                yaw = yaw + r.randrange(-randomConst,randomConst,10)/1000
                pitch = pitch + r.randrange(-randomConst2,randomConst2,10)/1000
                roll = roll + r.randrange(-randomConst3,randomConst3,10)/1000
            
            # orientationVector = np.asarray([0,0,0])
            # while (np.dot(orientationVector, [0,0,0])>0): # not sure if this works
            #     pitch = r.uniform(-3.14159,3.14159) 
            #     yaw = r.uniform(-3.14159,3.14159) 
            #     roll = r.uniform(-3.14159,3.14159)
            #     orientationVector = np.asarray([pitch,yaw,roll])

            links,joints = create_appendige(base,[x,y,z,roll,pitch,yaw],"arm_"+str(j))
            jointList.extend(joints)
            linksList.extend(links)
        robot = URDF("Model_"+str(i),links=linksList, joints=jointList)
        robot.save("Models/Model_"+str(i)+".urdf")

generateModelsStep(5,4,0,0,0)

In [113]:
for j in np.arange(0, 1.57 + .314, .314):
    output = 0
    for i in range(1,11):
        # print("iteration:",i)
        # print("current average MSE:",output[0]/i)
        # print("current average correlation:",output[1]/i)
        generateModelsStep(100,4,j*1000,j*1000,j*1000)
        bestModel = simulateModels(100, [[0.11071366557071198, 1.4306856382682968e-05, 0.00012585081957326537, 0.9938523372276092],[-0.1107156715211334, 5.1359029875874616e-05, 0.00012322408656520903, 0.9938521128707727],[0.00010303414843149941, -0.1107057358071606, 0.00013083062332979652, 0.9938532146784591],[7.442984885374358e-05, 0.11069317401764413, 0.00011865797329432217, 0.9938546179428772]],4)
        output += bestModel[1]
    print("MSE and pearson correlation for range: +/-",j)
    print(output/10.0)

MSE and pearson correlation for range: +/- 0.0
0.004127861114511311
MSE and pearson correlation for range: +/- 0.314
0.007968522757682122
MSE and pearson correlation for range: +/- 0.628
0.011480300877015234
MSE and pearson correlation for range: +/- 0.942
0.013044922825051545
MSE and pearson correlation for range: +/- 1.256
0.014642119206336898
MSE and pearson correlation for range: +/- 1.57
0.01106454982095082


https://urdfpy.readthedocs.io/en/latest/generated/urdfpy.Link.html#urdfpy.Link

https://urdfpy.readthedocs.io/en/latest/examples/index.html#loading-from-a-file
